In [16]:
def nfa_to_dfa(nfa_data):
    dfa_states = 2 ** nfa_data["states"]
    dfa_letters = nfa_data["letters"]
    dfa_start = nfa_data["start"]
    dfa_t_func = []
    dfa_final = []
    dfa_list = []
    q = []

    q.append((dfa_start,))

    nfa_transitions = {}
    dfa_transitions = {}

    # Store NFA transitions in a dictionary
    for transition in nfa_data["t_func"]:
        nfa_transitions[(transition[0], transition[1])] = transition[2]

    # Process each state in the DFA
    for in_state in q:
        for symbol in dfa_letters:
            # Single state transitions
            if len(in_state) == 1 and (in_state[0], symbol) in nfa_transitions:
                dfa_transitions[(in_state, symbol)] = nfa_transitions[(in_state[0], symbol)]

                if tuple(dfa_transitions[(in_state, symbol)]) not in q:
                    q.append(tuple(dfa_transitions[(in_state, symbol)]))
            else:
                dest = []
                f_dest = []

                # Handle transitions from multiple states (for combined DFA states)
                for n_state in in_state:
                    if (n_state, symbol) in nfa_transitions and nfa_transitions[(n_state, symbol)] not in dest:
                        dest.append(nfa_transitions[(n_state, symbol)])

                if dest:
                    for d in dest:
                        for value in d:
                            if value not in f_dest:
                                f_dest.append(value)

                    dfa_transitions[(in_state, symbol)] = f_dest

                    if tuple(f_dest) not in q:
                        q.append(tuple(f_dest))

    # Build the DFA transition function
    for key, value in dfa_transitions.items():
        temp_list = [[key[0], key[1], value]]
        dfa_t_func.extend(temp_list)

    # Determine final states for the DFA
    for q_state in q:
        for f_state in nfa_data["final"]:
            if f_state in q_state:
                dfa_final.append(q_state)

    # Construct the DFA as an OrderedDict
    dfa = OrderedDict()
    dfa["states"] = dfa_states
    dfa["letters"] = dfa_letters
    dfa["t_func"] = dfa_t_func
    dfa["start"] = dfa_start
    dfa["final"] = dfa_final

    return dfa



In [ ]:
import graphviz

def visualize_dfa(dfa, filename="dfa_graph"):
    dot = graphviz.Digraph(name=filename)

    # Set graph attributes
    dot.attr(rankdir='LR')  # Left to right orientation for DFA

    # Add states (nodes) to the graph
    for state in range(dfa["states"]):
        if tuple([state]) in dfa["final"]:  # Final states
            dot.node(str(state), shape='doublecircle')
        else:
            dot.node(str(state), shape='circle')

    # Add transitions (edges) to the graph
    for transition in dfa["t_func"]:
        start_state, symbol, next_state = transition
        dot.edge(str(start_state[0]), str(next_state[0]), label=symbol)

    # Mark the start state with an incoming arrow
    dot.node("", shape="none", width="0", height="0")
    dot.edge("", str(dfa["start"][0]))

    # Render and visualize the DFA
    dot.render(filename, format="png", cleanup=True)
    return dot


In [19]:
from collections import OrderedDict, defaultdict

def minimize_dfa(dfa):
    # Step 1: Remove unreachable states
    reachable = set()
    new_transitions = defaultdict(dict)
    queue = [dfa["start"]]

    while queue:
        state = queue.pop()
        if state not in reachable:
            reachable.add(state)
            for letter in dfa["letters"]:
                if (state, letter) in [tuple(trans[:2]) for trans in dfa["t_func"]]:
                    next_state = [trans[2] for trans in dfa["t_func"] if trans[0] == state and trans[1] == letter][0]
                    queue.append(tuple(next_state))
                    new_transitions[state][letter] = next_state

    # Remove unreachable states from transition function
    dfa["t_func"] = [[state, letter, next_state] for state, trans in new_transitions.items()
                     for letter, next_state in trans.items()]

    # Remove unreachable final states
    dfa["final"] = [f for f in dfa["final"] if f in reachable]

    # Step 2: Minimize the DFA using partition refinement
    partition = [set(dfa["final"]), set(reachable) - set(dfa["final"])]

    def distinguishable(state1, state2):
        for letter in dfa["letters"]:
            t1 = tuple([trans[2] for trans in dfa["t_func"] if trans[0] == state1 and trans[1] == letter])
            t2 = tuple([trans[2] for trans in dfa["t_func"] if trans[0] == state2 and trans[1] == letter])
            if t1 != t2:
                return True
        return False

    # Refine partition until no further splitting can be done
    while True:
        new_partition = []
        for group in partition:
            marked = set()
            unmarked = set()
            while group:
                state = group.pop()
                unmarked.add(state)
                for s in list(group):
                    if distinguishable(state, s):
                        marked.add(s)
                        group.remove(s)
                    else:
                        unmarked.add(s)
            if marked:
                new_partition.append(marked)
            new_partition.append(unmarked)
        if new_partition == partition:
            break
        partition = new_partition

    # Step 3: Merge equivalent states and create new DFA
    new_states = {}
    for group in partition:
        representative = tuple(group)[0]
        for state in group:
            new_states[state] = representative

    # Create the minimized DFA
    minimized_dfa = OrderedDict()
    minimized_dfa["states"] = len(partition)
    minimized_dfa["letters"] = dfa["letters"]
    minimized_dfa["start"] = new_states[dfa["start"]]
    minimized_dfa["final"] = list(set(new_states[f] for f in dfa["final"]))

    minimized_transitions = []
    for state, letter, next_state in dfa["t_func"]:
        minimized_transitions.append([new_states[state], letter, new_states[tuple(next_state)]])
    minimized_dfa["t_func"] = minimized_transitions

    return minimized_dfa


In [ ]:
import json
from collections import OrderedDict




# Load NFA input from a JSON file
with open('input.json') as file:
    nfa_data = json.load(file)


# Convert NFA to DFA
dfa_result = nfa_to_dfa(nfa_data)

visualize_dfa(dfa, "dfa_graph")

# Save DFA result to a JSON file
with open('output.json', 'w+') as output_file:
    json.dump(dfa_result, output_file, separators=(',', ':'))


minimized_dfa = minimize_dfa(dfa_result)

# Save the minimized DFA result to a JSON file
with open('minimized_output.json', 'w+') as output_file:
    json.dump(minimized_dfa, output_file, separators=(',', ':'))

